## ewf-ext-02-02-01 Oil sheen of natural oil seepage with Sentinel-1

### <a name="service">Service definition

In [1]:
service = dict([('title', 'ewf-ext-02-02-01 Oil sheen of natural oil seepage with Sentinel-1'),
                ('abstract', 'ewf-ext-02-02-01 Oil sheen of natural oil seepage with Sentinel-1'),
                ('id', 'ewf-ext-02-02-01')])

### <a name="parameter">Parameter Definition

In [2]:
backgroundWindowSize = dict([('id', 'backgroundWindowSize'),
                             ('value', '181'),
                             ('title', 'Background Window Size'),
                             ('abstract', 'Background Window Size: The window size in pixels for computing local mean backscatter level.'),
                             ('minOccurs', '1')])

In [3]:
thresholdShift = dict([('id', 'thresholdShift'),
                       ('value', '2.0'),
                       ('title', 'Threshold Shift'),
                       ('abstract', 'Threshold Shift (dB): The detecting threshold is lower than the local mean backscatter level by this amount.'),
                       ('minOccurs', '1')])

In [4]:
minClusterSizeInKm2 = dict([('id', 'minClusterSizeInKm2'),
                            ('value', '0.1'),
                            ('title', 'minClusterSizeInKm2'),
                            ('abstract', 'The minimum cluster size in square kilometer. Cluster with size smaller than this size is eliminated.'),
                            ('minOccurs', '1')])

In [5]:
regionOfInterest = dict([('id', 'regionOfInterest'),
                         ('value', 'POLYGON((-54.333 71.558, -49.843 71.558, -49.843 69.713, -54.333 69.713, -54.333 71.558))'),
                         ('title', 'WKT Polygon for the Region of Interest'),
                         ('abstract', 'Set the value of WKT Polygon')])

### <a name="runtime">Runtime parameter definition

**Input identifiers**

This is the Sentinel-1 stack of products' identifiers

In [6]:
input_identifiers = ['S1B_IW_GRDH_1SDH_20171025T204653_20171025T204718_007991_00E1E2_CBD2']

**Input references**

This is the Sentinel-1 stack catalogue references

In [7]:
input_references = ['https://catalog.terradue.com/sentinel1/search?uid=S1B_IW_GRDH_1SDH_20171025T204653_20171025T204718_007991_00E1E2_CBD2']

**Data path**

This path defines where the data is staged-in.

In [8]:
data_path = "/workspace/data"

**Aux folders**

In [9]:
output_folder = ''
temp_folder = 'temp'

### <a name="workflow">Workflow

#### Import the packages required for processing the data

In [10]:
import snappy

from snappy import jpy
from snappy import ProductIO
from snappy import GPF
from snappy import HashMap

import numpy as np
from osgeo import gdal, ogr, osr

import os
import shutil

import gc

import datetime

import cioppy
ciop = cioppy.Cioppy() 

#### Methods

In [11]:
# remove contents of a given folder
# used to clean a temporary folder
def rm_cfolder(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path): shutil.rmtree(file_path)
        except Exception as e:
            print(e)
            
def write_output_image(filepath, output_matrix, image_format, data_format, mask=None, output_projection=None, output_geotransform=None, no_data_value=None):
    
    driver = gdal.GetDriverByName(image_format)
    out_rows = np.size(output_matrix, 0)
    out_columns = np.size(output_matrix, 1)
    
    
    if mask is not None and mask is not 0:
        # TODO: check if output folder exists
        output = driver.Create(filepath, out_columns, out_rows, 2, data_format)
        mask_band = output.GetRasterBand(2)
        mask_band.WriteArray(mask)
        if no_data_value is not None:
            output_matrix[mask > 0] = no_data_value
    else:
        output = driver.Create(filepath, out_columns, out_rows, 1, data_format)
    
    if output_projection is not None:
        output.SetProjection(output_projection)
    if output_geotransform is not None:
        output.SetGeoTransform(output_geotransform)
    
    raster_band = output.GetRasterBand(1)
    if no_data_value is not None:
        raster_band.SetNoDataValue(no_data_value)
    raster_band.WriteArray(output_matrix)
    
    gdal.Warp(filepath, output, format="GTiff", outputBoundsSRS='EPSG:4326', xRes=output_geotransform[1], yRes=-output_geotransform[5], targetAlignedPixels=True)
    
    
def get_formatted_date(datetime_str):
    date = datetime.datetime.strftime(datetime_str, '%Y-%m-%dT%H:%M:%SZ')
    return date


def write_properties_file(output_name, first_date, last_date, region_of_interest):
    
    title = 'Output %s' % output_name
    
    first_date = datetime.datetime(year=first_date.year, month=first_date.month, day=first_date.day)
    first_date = first_date + datetime.timedelta(days=0, hours=0, minutes=0, seconds=0)
    first_date = get_formatted_date(first_date)
    
    last_date = datetime.datetime(year=last_date.year, month=last_date.month, day=last_date.day)
    last_date = last_date + datetime.timedelta(days=0, hours=23, minutes=59, seconds=59)
    last_date = get_formatted_date(last_date)

    with open(output_name + '.properties', 'wb') as file:
        file.write('title=%s\n' % title)
        file.write('date=%s/%s\n' % (first_date, last_date))
        file.write('geometry=%s' % (region_of_interest))

In [12]:
check_results = False

#### Read the products

In [ ]:
if len(output_folder) > 0:
    if not os.path.isdir(output_folder):
        os.mkdir(output_folder)

if not os.path.isdir(temp_folder):
    os.mkdir(temp_folder)

In [ ]:
s1meta = "manifest.safe"

products = []

for s1path in input_identifiers:

    s1prd= "%s/%s/%s.SAFE/%s" % (data_path, s1path, s1path, s1meta)
    reader = ProductIO.getProductReader("SENTINEL-1")
    product = reader.readProductNodes(s1prd, None)
    products.append(product)

#### Extract information about the Sentinel-1 GRD products:

In [ ]:
for product in products:

    width = product.getSceneRasterWidth()
    height = product.getSceneRasterHeight()
    name = product.getName()
    band_names = product.getBandNames()
    print("Product: %s, %d x %d pixels" % (name, width, height))
    print("Bands:   %s" % (list(band_names)))
    

#### Process the data

##### Check number of products

In [ ]:
nProducts = len(products)

if nProducts == 1:
    
    product = products[0]
    
else:
    pass # TODO: merge

##### Subset

In [ ]:
#aoi = 'POLYGON((-54.333 71.559, -49.842 71.559, -49.842 69.713, -54.333 69.713, -54.333 71.559))'
#aoi = 'POLYGON((-54.333 71.558, -49.843 71.558, -49.843 69.713, -54.333 69.713, -54.333 71.558))'
aoi = regionOfInterest['value']

In [ ]:
WKTReader = snappy.jpy.get_type('com.vividsolutions.jts.io.WKTReader')

geom = WKTReader().read(aoi);


HashMap = jpy.get_type('java.util.HashMap')    
GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()

parameters = HashMap()
parameters.put('copyMetadata', True)
parameters.put('geoRegion', geom)

subset = GPF.createProduct('Subset', parameters, product)

##### Calibration

In [ ]:
parameters = HashMap()

parameters.put('auxFile', 'Latest Auxiliary File')
parameters.put('outputSigmaBand', True)
#parameters.put('selectedPolarisations', 'HH')

calibrate = GPF.createProduct('Calibration', parameters, subset)

##### Speckle-Filter

In [ ]:
parameters = HashMap()

parameters.put('filter','Lee')
parameters.put('filterSizeX','3')
parameters.put('filterSizeY','3')
parameters.put('dampingFactor','2')
parameters.put('estimateENL',True)
parameters.put('enl','1.0')
parameters.put('numLooksStr','1')
parameters.put('windowSize','7x7')
parameters.put('targetWindowSizeStr','3x3')
parameters.put('sigmaStr','0.9')
parameters.put('anSize','50')

filtered = GPF.createProduct('Speckle-Filter', parameters, calibrate)

##### Land-Sea-Mask

In [ ]:
parameters = HashMap()

parameters.put('vectorFile', '/workspace/dev/ewf-ext-02-02-01/src/main/app-resources/notebook/libexec/land_vector/land.shp')
parameters.put('separateShapes', False)

addvector = GPF.createProduct('Import-Vector', parameters, filtered)

In [ ]:
parameters = HashMap()

parameters.put('landMask', False)
parameters.put('useSRTM', False)
parameters.put('geometry', 'land')
parameters.put('invertGeometry', True)
parameters.put('shorelineExtension', 10)

landMasked = GPF.createProduct('Land-Sea-Mask', parameters, addvector)

##### Oil-Spill-Detection

In [ ]:
var1 = int(backgroundWindowSize['value'])
var2 = float(thresholdShift['value'])

parameters = HashMap()

parameters.put('sourceBands', 'Sigma0_HH,Sigma0_HV')
parameters.put('backgroundWindowSize', var1)
parameters.put('k', var2)

oildetection = GPF.createProduct('Oil-Spill-Detection', parameters, landMasked)

#####  Oil-Spill-Clustering

In [ ]:
var1 = float(minClusterSizeInKm2['value'])

parameters = HashMap()

parameters.put('minClusterSizeInKm2', var1)

oilclust = GPF.createProduct('Oil-Spill-Clustering', parameters, oildetection)
name = oilclust.getName()
timestamp = name.split("_")[5] 
date = timestamp[:8]

In [ ]:
#dir(oilclust)

In [ ]:
band_names = list(oilclust.getBandNames())
print(band_names)

In [ ]:
if check_results:
    import numpy as np
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.colors as colors
    import gc 

    %matplotlib inline

    def plotBand(product, band, vmin, vmax):
     
        band = product.getBand(band)

        w = band.getRasterWidth()
        h = band.getRasterHeight()

        band_data = np.zeros(w * h, np.float32)
        band.readPixels(0, 0, w, h, band_data)

        band_data.shape = h, w

        imgplot = plt.imshow(band_data, cmap=plt.cm.binary_r, vmin=vmin, vmax=vmax)

    
        return imgplot 


    fig = plt.figure(figsize=(20,20))
    i = 1

    a=fig.add_subplot(330+i)
    imgplot = plotBand(oilclust, 'Sigma0_HH', -25, 5)
    name = oilclust.getName()
    timestamp = name.split("_")[5] 
    date = timestamp[:8]
    a.set_title(date)
    i = i+1

    plt.tight_layout()
    fig = plt.gcf()
    plt.show()

    fig.clf()
    plt.close()
    gc.collect()

In [ ]:
#oilclust.getName()

#####  Garbage collector

In [ ]:
subset = None
addvector = None
landMasked = None
calibrate = None
oildetection = None
#oilclust = None

gc.collect()

#####  Save temporary result

In [ ]:
message = 'First part ...' 
ciop.log('INFO', message)

output_path = os.path.join(temp_folder, oilclust.getName())

ProductIO.writeProduct(oilclust, output_path, 'BEAM-DIMAP')

In [ ]:
oilclust = None

gc.collect()

#####  Ellipsoid-Correction-RD

In [ ]:
poildect = ProductIO.readProduct(output_path + '.dim')

In [ ]:
parameters = HashMap()

parameters.put('sourceBands', 'Sigma0_HH_oil_spill_bit_msk')

parameters.put('demName', 'GETASSE30')
parameters.put('imgResamplingMethod', 'BILINEAR_INTERPOLATION')

proj = '''GEOGCS["WGS84(DD)", DATUM["WGS84", SPHEROID["WGS84", 6378137.0, 298.257223563]], PRIMEM["Greenwich", 0.0], UNIT["degree", 0.017453292519943295], AXIS["Geodetic longitude", EAST], AXIS["Geodetic latitude", NORTH]]'''

parameters.put('mapProjection', proj)       # comment this line if no need to convert to UTM/WGS84, default is WGS84parameters.put('saveProjectedLocalIncidenceAngle', True)
parameters.put('saveSelectedSourceBand', True)

parameters.put('pixelSpacingInMeter', 25.0)

output = GPF.createProduct('Ellipsoid-Correction-RD', parameters, poildect)

In [ ]:
message = 'Second part ...' 
ciop.log('INFO', message)

output_path = os.path.join(temp_folder, output.getName() + '.tif')

ProductIO.writeProduct(output, output_path, 'GeoTIFF')

In [ ]:
output = None

gc.collect()

#####  Save tiff file and properties file

In [ ]:
dataset = gdal.Open(output_path)

product_array = dataset.GetRasterBand(1).ReadAsArray()
projection = dataset.GetProjection()
geotransform = dataset.GetGeoTransform()
no_data_value = dataset.GetRasterBand(1).GetNoDataValue()
data_type = dataset.GetRasterBand(1).DataType

In [ ]:
message = 'Last part ...' 
ciop.log('INFO', message)

# date in str
date_str = input_identifiers[0].split('_')[4].split('T')[0]

# date in datetime
dt = datetime.datetime.strptime(date_str, "%Y%m%d")

# output file path
output_tiff_path = os.path.join(output_folder, 'oil_sheen_bit_S1_HH_' + date_str + '.tif')

# writes final tiff image
write_output_image(output_tiff_path, product_array, 'GTiff', 1, mask=None, output_projection=projection, output_geotransform=geotransform, no_data_value=0)

# writes properties file
write_properties_file(output_tiff_path, dt, dt, aoi)

#####  Remove temporary folder

In [ ]:
shutil.rmtree(temp_folder)